In [17]:
from dotenv import load_dotenv
import requests
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# plt.style.use('classic')
%matplotlib inline

import seaborn as sns
# sns.set()
# plt.rcParams['figure.figsize'] = (31,7)

In [2]:
cacheDir = '~/pandas_cache'

load_dotenv()
HASURA_SECRET = os.getenv("HASURA_SECRET")

In [3]:
def run_query(query, limit=1000000, offset=None):
    
    if offset is None:
        return_df = None
        offset = 0;
        while True:
            df = run_query(query, limit, offset)
            if return_df is None:
                return_df = df.copy()
            else:
                return_df = pd.concat([return_df, df], ignore_index=True)
            print(f"{len(return_df)} rows")
            if len(df) < limit:
                break
            else:
                offset += limit
        return return_df        
    
    url = 'http://localhost:8080/v1/query'
    headers = {"content-type": "application/json", 
               "x-hasura-admin-secret": HASURA_SECRET
              }
    try:
        body = {
            "type": "run_sql",
            "args": {
                "sql": f"{query} limit {limit} offset {offset}"
            }
        }
        r = requests.post(url, json=body, headers=headers)
        results = json.loads(r.text)["result"]
        return pd.DataFrame(results[1:], columns=results[0])
    except Exception as error:
        print(error)
        print(r.status_code)
        print(r.text)

In [4]:
if not os.path.exists(cacheDir):
  os.makedirs(cacheDir)

def cache_query(name, query, munger=None, use_cache=True):
    fname = f"{cacheDir}/{name}.pkl"
    try:
        if not use_cache:
            raise Exception("no cache")
        df = pd.read_pickle(fname)
        print("read from cache")
        print(f"{len(df)} rows")
        return df
    except:
        print("running database query")
        df = run_query(query)
        df.to_pickle(fname)
        if munger:
            df = munger(df)
        df.to_pickle(fname)
        return df 

In [5]:
def query_year(year):
    return f"""
select 
    n.created_at, n.sent_at, n.updated_at, n.notification_status as status,
    n.notification_type as type,
    extract(epoch from (n.sent_at - n.created_at)) as processing_seconds,
    extract(epoch from (n.updated_at - n.sent_at)) as callback_seconds,
    t.process_type as priority,
    s.count_as_live,
    s.name
from 
    notification_history n
    join services s on n.service_id = s.id
    join templates_history t on n.template_id = t.id and n.template_version = t.version
where
    s.count_as_live and
    extract(YEAR from n.created_at) = {year}
order by created_at
"""

In [6]:
def munge_df(df):
    print("munging data")
    df = df.replace("NULL", np.nan)
    df['created_at'] = pd.to_datetime(df.created_at)
    df['updated_at'] = pd.to_datetime(df.updated_at)
    df['date'] = df.created_at.dt.date
    df['month'] = df.created_at.dt.to_period('M')
    df['processing_seconds'] = pd.to_numeric(df.processing_seconds,  errors='coerce')
    df['callback_seconds'] = pd.to_numeric(df.callback_seconds,  errors='coerce')
    return df

## Get data

query / munging times:
* 2020: 8min 36s
* 2021: 31min 40s
* 2022: 31min 43s

In [7]:
%time df = cache_query('notification_history_2022', query_year(2022), munger=munge_df)

read from cache
16824277 rows
CPU times: user 15.7 s, sys: 4.88 s, total: 20.6 s
Wall time: 21.3 s


## Monthly amount of notifications

In [8]:
month_counts = df[['month']].value_counts()
pd.DataFrame(month_counts).sort_values(by='month')

,0
month,
2022-01,2300506
2022-02,2410295
2022-03,4011291
2022-04,4930607
2022-05,3165550
2022-06,6028


## By type

In [28]:
df_counts = df[['month', 'type', 'processing_seconds']].groupby(['month', 'type']).count()
df_counts.columns = ['count']
df_counts = df_counts.reset_index()
df_counts['percent'] = df_counts.apply(lambda r: r['count'] / month_counts[r['month']] * 100, axis=1).round(1)
df_counts.set_index(['month', 'type'])

count  percent
month   type                   
2022-01 email  2234721     97.1
        sms      65783      2.9
2022-02 email  2337695     97.0
        sms      72594      3.0
2022-03 email  3905388     97.4
        sms     105894      2.6
2022-04 email  4807629     97.5
        sms     122943      2.5
2022-05 email  3071041     97.0
        sms      94466      3.0
2022-06 email     1504     25.0
        sms       4524     75.0

In [29]:
print(df_counts.to_markdown(index=False))

| month   | type   |   count |   percent |
|:--------|:-------|--------:|----------:|
| 2022-01 | email  | 2234721 |      97.1 |
| 2022-01 | sms    |   65783 |       2.9 |
| 2022-02 | email  | 2337695 |      97   |
| 2022-02 | sms    |   72594 |       3   |
| 2022-03 | email  | 3905388 |      97.4 |
| 2022-03 | sms    |  105894 |       2.6 |
| 2022-04 | email  | 4807629 |      97.5 |
| 2022-04 | sms    |  122943 |       2.5 |
| 2022-05 | email  | 3071041 |      97   |
| 2022-05 | sms    |   94466 |       3   |
| 2022-06 | email  |    1504 |      25   |
| 2022-06 | sms    |    4524 |      75   |


## By priority

In [30]:
df_counts = df[['month', 'priority', 'processing_seconds']].groupby(['month', 'priority']).count()
df_counts.columns = ['count']
df_counts = df_counts.reset_index()
df_counts['percent'] = df_counts.apply(lambda r: r['count'] / month_counts[r['month']] * 100, axis=1).round(1)
df_counts.set_index(['month', 'priority'])

count  percent
month   priority                  
2022-01 bulk       424493     18.5
        normal    1859685     80.8
        priority    16326      0.7
2022-02 bulk       523494     21.7
        normal    1862528     77.3
        priority    24267      1.0
2022-03 bulk      1695289     42.3
        normal    2269286     56.6
        priority    46707      1.2
2022-04 bulk      2406941     48.8
        normal    2456123     49.8
        priority    67508      1.4
2022-05 bulk       480625     15.2
        normal    2302800     72.7
        priority   382082     12.1
2022-06 normal       6028    100.0

In [31]:
print(df_counts.to_markdown(index=False))

| month   | priority   |   count |   percent |
|:--------|:-----------|--------:|----------:|
| 2022-01 | bulk       |  424493 |      18.5 |
| 2022-01 | normal     | 1859685 |      80.8 |
| 2022-01 | priority   |   16326 |       0.7 |
| 2022-02 | bulk       |  523494 |      21.7 |
| 2022-02 | normal     | 1862528 |      77.3 |
| 2022-02 | priority   |   24267 |       1   |
| 2022-03 | bulk       | 1695289 |      42.3 |
| 2022-03 | normal     | 2269286 |      56.6 |
| 2022-03 | priority   |   46707 |       1.2 |
| 2022-04 | bulk       | 2406941 |      48.8 |
| 2022-04 | normal     | 2456123 |      49.8 |
| 2022-04 | priority   |   67508 |       1.4 |
| 2022-05 | bulk       |  480625 |      15.2 |
| 2022-05 | normal     | 2302800 |      72.7 |
| 2022-05 | priority   |  382082 |      12.1 |
| 2022-06 | normal     |    6028 |     100   |


## Quantiles

In [57]:
df_p = df[['month', 'priority', 'processing_seconds']]
df_50 = df_p.groupby(['month', 'priority']).quantile(q=0.5).apply(np.ceil)
df_50.columns = ['processing_50']
df_90 = df_p.groupby(['month', 'priority']).quantile(q=0.9).apply(np.ceil)
df_90.columns = ['processing_90']
df_99 = df_p.groupby(['month', 'priority']).quantile(q=0.99).apply(np.ceil)
df_99.columns = ['processing_99']
processing_df = pd.concat([df_50, df_90, df_99], axis=1).reset_index()
processing_df.set_index(['month', 'priority'])

processing_50  processing_90  processing_99
month   priority                                             
2022-01 bulk               20.0           49.0           68.0
        normal              7.0           34.0           78.0
        priority            1.0            1.0            5.0
2022-02 bulk               18.0          881.0         2615.0
        normal              6.0           35.0           79.0
        priority            1.0            2.0            5.0
2022-03 bulk              897.0         2188.0         2935.0
        normal              4.0          954.0         3157.0
        priority            1.0            1.0            4.0
2022-04 bulk             1238.0         2608.0         3374.0
        normal              3.0          506.0         1179.0
        priority            1.0            2.0            5.0
2022-05 bulk              350.0         1510.0         2351.0
        normal             12.0          966.0         1677.0
        priority            2.0            8.0           20.0
2022-06 normal              7.0          252.0          577.0

## Email Quantiles

In [58]:
df_p = df.query("type=='email'")[['month', 'priority', 'processing_seconds']]
df_50 = df_p.groupby(['month', 'priority']).quantile(q=0.5).apply(np.ceil)
df_50.columns = ['processing_50']
df_90 = df_p.groupby(['month', 'priority']).quantile(q=0.9).apply(np.ceil)
df_90.columns = ['processing_90']
df_99 = df_p.groupby(['month', 'priority']).quantile(q=0.99).apply(np.ceil)
df_99.columns = ['processing_99']
processing_df = pd.concat([df_50, df_90, df_99], axis=1).reset_index()
processing_df.set_index(['month', 'priority'])

processing_50  processing_90  processing_99
month   priority                                             
2022-01 bulk               20.0           49.0           68.0
        normal              7.0           34.0           73.0
        priority            1.0            1.0            2.0
2022-02 bulk               18.0          881.0         2615.0
        normal              6.0           34.0           71.0
        priority            1.0            1.0            3.0
2022-03 bulk              897.0         2188.0         2935.0
        normal              4.0          972.0         3180.0
        priority            1.0          527.0          644.0
2022-04 bulk             1238.0         2608.0         3374.0
        normal              3.0          508.0         1144.0
        priority            1.0            2.0           11.0
2022-05 bulk              350.0         1510.0         2351.0
        normal             12.0          974.0         1685.0
        priority            2.0            9.0           20.0
2022-06 normal              1.0            8.0           21.0

In [59]:
print(processing_df.to_markdown(index=False))

| month   | priority   |   processing_50 |   processing_90 |   processing_99 |
|:--------|:-----------|----------------:|----------------:|----------------:|
| 2022-01 | bulk       |              20 |              49 |              68 |
| 2022-01 | normal     |               7 |              34 |              73 |
| 2022-01 | priority   |               1 |               1 |               2 |
| 2022-02 | bulk       |              18 |             881 |            2615 |
| 2022-02 | normal     |               6 |              34 |              71 |
| 2022-02 | priority   |               1 |               1 |               3 |
| 2022-03 | bulk       |             897 |            2188 |            2935 |
| 2022-03 | normal     |               4 |             972 |            3180 |
| 2022-03 | priority   |               1 |             527 |             644 |
| 2022-04 | bulk       |            1238 |            2608 |            3374 |
| 2022-04 | normal     |               3 |          

## SMS Quantiles

In [60]:
df_p = df.query("type=='sms'")[['month', 'priority', 'processing_seconds']]
df_50 = df_p.groupby(['month', 'priority']).quantile(q=0.5).apply(np.ceil)
df_50.columns = ['processing_50']
df_90 = df_p.groupby(['month', 'priority']).quantile(q=0.9).apply(np.ceil)
df_90.columns = ['processing_90']
df_99 = df_p.groupby(['month', 'priority']).quantile(q=0.99).apply(np.ceil)
df_99.columns = ['processing_99']
processing_df = pd.concat([df_50, df_90, df_99], axis=1).reset_index()
processing_df.set_index(['month', 'priority'])

processing_50  processing_90  processing_99
month   priority                                             
2022-01 normal             17.0          118.0          451.0
        priority            1.0            1.0            5.0
2022-02 normal             19.0          212.0          557.0
        priority            1.0            2.0            5.0
2022-03 normal             16.0          345.0         1326.0
        priority            1.0            1.0            4.0
2022-04 normal             23.0          381.0         3271.0
        priority            1.0            2.0            5.0
2022-05 normal              8.0          239.0          575.0
        priority            1.0            2.0            6.0
2022-06 normal             12.0          327.0          580.0

In [61]:
print(processing_df.to_markdown(index=False))

| month   | priority   |   processing_50 |   processing_90 |   processing_99 |
|:--------|:-----------|----------------:|----------------:|----------------:|
| 2022-01 | normal     |              17 |             118 |             451 |
| 2022-01 | priority   |               1 |               1 |               5 |
| 2022-02 | normal     |              19 |             212 |             557 |
| 2022-02 | priority   |               1 |               2 |               5 |
| 2022-03 | normal     |              16 |             345 |            1326 |
| 2022-03 | priority   |               1 |               1 |               4 |
| 2022-04 | normal     |              23 |             381 |            3271 |
| 2022-04 | priority   |               1 |               2 |               5 |
| 2022-05 | normal     |               8 |             239 |             575 |
| 2022-05 | priority   |               1 |               2 |               6 |
| 2022-06 | normal     |              12 |          

## Busiest 10 minutes

In [12]:
df['ten_minutes'] = df.created_at.dt.ceil('10min')
df2 = pd.DataFrame(df[['month', 'ten_minutes']].value_counts()).reset_index()
df2.columns = ['month', 'ten_minutes', 'count']
df2 = df2.sort_values(by=['month', 'count']).reset_index(drop=True)
df2.groupby('month').last()

,ten_minutes,count
month,,
2022-01,2022-01-11 19:10:00,20692
2022-02,2022-02-16 19:00:00,51752
2022-03,2022-03-08 18:20:00,100158
2022-04,2022-04-13 12:10:00,95040
2022-05,2022-05-04 03:10:00,51498
2022-06,2022-06-01 01:40:00,375


## Find busiest hour from 1 minute segments

In [13]:
df['one_minute'] = df.created_at.dt.ceil('1min')
df2 = pd.DataFrame(df[['month', 'one_minute']].value_counts()).reset_index()
df2.columns = ['month', 'hour_ending_at', 'count']

df2 = df2.sort_values(by=['hour_ending_at'])

df2['count'] = df2[['count']].rolling(window=60).sum()
df2 = df2.dropna()

df2.sort_values(by=['count']).groupby('month').last()

,hour_ending_at,count
month,,
2022-01,2022-01-21 19:49:00,81380.0
2022-02,2022-02-09 22:35:00,100812.0
2022-03,2022-03-08 19:09:00,146253.0
2022-04,2022-04-21 16:58:00,125513.0
2022-05,2022-05-09 15:26:00,99034.0
2022-06,2022-06-03 03:42:00,1038.0
